# Eat Safe, Love

## Notebook Set Up

In [11]:
!mongoimport --type json -d uk_food -c establishments --drop --jsonArray establishments.json

'mongoimport' is not recognized as an internal or external command,
operable program or batch file.


In [44]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [69]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
from pprint import pprint

In [53]:
engine = create_engine('sqlite:///establishments.db')

In [54]:
Session = sessionmaker(bind=engine)

In [55]:
# Create a session
session = Session()

In [56]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [57]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [58]:
# review the collections in our database
# review the collections in our new database
collections = db.list_collection_names()
print("Collections in the database:", collections)

Collections in the database: ['establishments']


In [59]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [78]:
print(mongo.list_database_names())

['admin', 'config', 'local', 'uk_food']


In [80]:
# Find the establishments with a hygiene score of 20
query = establishments.find({"scores.Hygiene": 20})

# Use count_documents to display the number of documents in the result
print("Number of documents in result:", establishments.count_documents({"scores.Hygiene": 20}))

# Display the first document in the results using pprint
pprint(query[0])

Number of documents in result: 41
{'AddressLine1': '5-6 Southfields Road',
 'AddressLine2': 'Eastbourne',
 'AddressLine3': 'East Sussex',
 'AddressLine4': '',
 'BusinessName': 'The Chase Rest Home',
 'BusinessType': 'Caring Premises',
 'BusinessTypeID': 5,
 'ChangesByServerID': 0,
 'Distance': 4613.888288172291,
 'FHRSID': 110681,
 'LocalAuthorityBusinessID': '4029',
 'LocalAuthorityCode': '102',
 'LocalAuthorityEmailAddress': 'Customerfirst@eastbourne.gov.uk',
 'LocalAuthorityName': 'Eastbourne',
 'LocalAuthorityWebSite': 'http://www.eastbourne.gov.uk/foodratings',
 'NewRatingPending': False,
 'Phone': '',
 'PostCode': 'BN21 1BU',
 'RatingDate': '2021-09-23T00:00:00',
 'RatingKey': 'fhrs_0_en-gb',
 'RatingValue': 0,
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('66c2bd06c367278fe4a11b89'),
 'geocode': {'latitude': '50.769705', 'longitude': '0.27694'},
 'links': [{'href': 'https://api.ratings.food.gov.uk/establishments/110681',
            'rel': 'self'}],
 'meta': {'dat

In [81]:
# Convert the result to a Pandas DataFrame
result_df = pd.DataFrame(list(query))
# Display the number of rows in the DataFrame
num_rows = len(result_df)
# Display the first 10 rows of the DataFrame
print(result_df.head(10))

                        _id   FHRSID  ChangesByServerID  \
0  66c2bd06c367278fe4a11b89   110681                  0   
1  66c2bd06c367278fe4a11f07   612039                  0   
2  66c2bd06c367278fe4a1220f   730933                  0   
3  66c2bd06c367278fe4a123fe   172735                  0   
4  66c2bd06c367278fe4a1240f   172953                  0   
5  66c2bd06c367278fe4a12dad   512854                  0   
6  66c2bd06c367278fe4a12fcd  1537089                  0   
7  66c2bd07c367278fe4a144f9   155648                  0   
8  66c2bd07c367278fe4a1493e  1012883                  0   
9  66c2bd08c367278fe4a1514d   644109                  0   

  LocalAuthorityBusinessID               BusinessName  \
0                     4029        The Chase Rest Home   
1                1970/FOOD                 Brenalwood   
2                1698/FOOD              Melrose Hotel   
3             PI/000023858              Seaford Pizza   
4             PI/000024532              Golden Palace   
5       

### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [86]:
import pprint

In [101]:
# Find the establishments with LocalAuthorityName containing 'London' and a RatingValue greater than or equal to 4.
query = {
    "LocalAuthorityName": {"$regex": "London"},
    "RatingValue": {"$gte": 4}
}

# Use count_documents to display the number of documents in the result
result_count = establishments.count_documents(query)
print("Number of documents in result:", result_count)

# Display the first document in the results using pprint
result_document = establishments.find(query)
print(result_document[0])

Number of documents in result: 33
{'_id': ObjectId('66c2bd07c367278fe4a1359f'), 'FHRSID': 621707, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000025307', 'BusinessName': "Charlie's", 'BusinessType': 'Other catering premises', 'BusinessTypeID': 7841, 'AddressLine1': 'Oak Apple Farm Building 103 Sheernes Docks', 'AddressLine2': 'Sheppy Kent', 'AddressLine3': '', 'AddressLine4': '', 'PostCode': 'ME12', 'Phone': '', 'RatingValue': 4, 'RatingKey': 'fhrs_4_en-gb', 'RatingDate': '2021-10-18T00:00:00', 'LocalAuthorityCode': '508', 'LocalAuthorityName': 'City of London Corporation', 'LocalAuthorityWebSite': 'http://www.cityoflondon.gov.uk/Corporation/homepage.htm', 'LocalAuthorityEmailAddress': 'publicprotection@cityoflondon.gov.uk', 'scores': {'Hygiene': 5, 'Structural': 10, 'ConfidenceInManagement': 5}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '0.508551', 'latitude': '51.369321'}, 'RightToReply': '', 'Distance': 4627.439467780196, 'NewRatingPending': False, 'meta': {'dataSou

In [ ]:
LocalAuthorityName

In [104]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)

result_df = pd.DataFrame(cursor)

# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(result_df))

# Display the first 10 rows of the DataFrame
result_df.head(10)

Number of rows in the DataFrame: 33


,_id,FHRSID,ChangesByServerID,LocalAuthorityBusinessID,BusinessName,BusinessType,BusinessTypeID,AddressLine1,AddressLine2,AddressLine3,...,LocalAuthorityWebSite,LocalAuthorityEmailAddress,scores,SchemeType,geocode,RightToReply,Distance,NewRatingPending,meta,links
0,66c2bd07c367278fe4a1359f,621707,0,PI/000025307,Charlie's,Other catering premises,7841,Oak Apple Farm Building 103 Sheernes Docks,Sheppy Kent,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 5, 'Structural': 10, 'ConfidenceIn...",FHRS,"{'longitude': '0.508551', 'latitude': '51.3693...",,4627.439468,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'https://api.ratings...."
1,66c2bd07c367278fe4a138c6,1130836,0,PI/000034075,Mv City Cruises Erasmus,Other catering premises,7841,Cherry Garden Pier,Cherry Garden Street Rotherhithe,Charlton,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.249255999922752', 'latitude':...",,4627.873179,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
2,66c2bd07c367278fe4a14414,293783,0,PI/000002614,Benfleet Motor Yacht Club,Other catering premises,7841,Ferry Road,South Benfleet Essex,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.561954', 'latitude': '51.5438...",,4631.968418,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
3,66c2bd08c367278fe4a15213,1315095,0,PI/000036464,Coombs Catering t/a The Lock and Key,Restaurant/Cafe/Canteen,1,Leslie Ford House,Tilbury,Charlton,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.346336990594864', 'latitude':...",,4636.846754,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
4,66c2bd08c367278fe4a15216,294474,0,PI/000014647,Tilbury Seafarers Centre,Restaurant/Cafe/Canteen,1,Tenants Row Tilbury Docks,Tilbury Essex,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.346325', 'latitude': '51.4640...",,4636.847650,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
5,66c2bd08c367278fe4a1591c,294900,0,PI/000019066,Mv Valulla,Other catering premises,7841,Reeds River Cruises LtdKings ReachRiver Thames...,London,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 0, 'ConfidenceInM...",FHRS,"{'longitude': '0.287555', 'latitude': '51.5040...",,4640.460834,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
6,66c2bd09c367278fe4a17a2a,293756,0,PI/000002538,Tereza Joanne,Other catering premises,7841,Funcraft UK Ltd King George V Dock Woolwich Ma...,London,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 0, 'Structural': 5, 'ConfidenceInM...",FHRS,"{'longitude': '0.069286', 'latitude': '51.5011...",,4648.301822,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.ratings.f..."
7,66c2bd09c367278fe4a17df1,878523,0,PI/000029844,Brick Lane Brews,Restaurant/Cafe/Canteen,1,Air side London City Airport,London,,...,http://www.cityoflondon.gov.uk/Corporation/hom...,publicprotection@cityoflondon.gov.uk,"{'Hygiene': 10, 'Structural': 5, 'ConfidenceIn...",FHRS,"{'longitude': '0.048006', 'latitude': '51.5037...",,4649.173485,False,"{'dataSource': None, 'extractDate': '0001-01-0...","[{'rel': 'self', 'href': 'http://api.rating

### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [131]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score
from pymongo import MongoClient
from pprint import pprint  # Import the pprint function

# Connect to MongoDB
mongo = MongoClient(port=27017)

# Assign the database and collection to variables
es_db = mongo['establishments']  # Replace with your database name
collection = es_db['RatingValue']  # Replace with your collection name

penang_lat = 51.49014200  # Example latitude
penang_lon = -0.08384000  # Example longitude

# Define the search range (0.01 degree on either side)
lat_range = [penang_lat - 0.01, penang_lat + 0.01]
lon_range = [penang_lon - 0.01, penang_lon + 0.01]

# Query the database
results = collection.find({
    "RatingValue": 5,
    "geocode.latitude": {"$gte": lat_range[0], "$lte": lat_range[1]},
    "geocode.longitude": {"$gte": lon_range[0], "$lte": lon_range[1]}
}).sort("scores.Hygiene", 1).limit(5)

# Print the results
for result in results:
    pprint(result)



In [132]:
# Convert result to Pandas DataFrame
# Assuming 'results' is your result data
result_df = pd.DataFrame(results)
result_df

""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [133]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
# Connect to MongoDB
mongo = MongoClient(port=27017)

# Assign the database and collection to variables
es_db = mongo['establishments']  # Replace with your database name
collection = es_db['establishments']  # Replace with your collection name

# Define the aggregation pipeline
pipeline = [
    # Step 1: Match establishments with a hygiene score of 0
    {"$match": {"scores.Hygiene": 0}},
    
    # Step 2: Group the matches by Local Authority
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    
    # Step 3: Sort the matches from highest to lowest
    {"$sort": {"count": -1}},
    
    # Step 4: Limit the results to the top 10
    {"$limit": 10}
]

# Run the aggregation pipeline
results = list(collection.aggregate(pipeline))

# Print the number of documents in the result
print(f"Number of Local Authorities with hygiene score of 0: {len(results)}")

# Print the first 10 results
pprint(results)

Number of Local Authorities with hygiene score of 0: 0
[]


In [134]:

# Convert the result to a Pandas DataFrame
df = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"\nNumber of rows in the DataFrame: {len(df)}")

# Display the first 10 rows of the DataFrame
print("\nFirst 10 rows of the DataFrame:")
print(df.head(10))


Number of rows in the DataFrame: 0

First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []
